# This project is designed for choosing a ideal place to open a Thai Restaurant.

In [11]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [146]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Solving environment: failed with current_repodata.json, will retry with next repodata source.
Solving environment: done

## Package Plan ##

  environment location: //anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.1.0               |           py37_0         722 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.6.16          |           py37_0         148 KB  conda-forge
    conda-4.7.10               |           py37_0         3.0 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.9 MB

The following NEW packages will b

In [2]:
pip install geopy

     |████████████████████████████████| 102kB 1.3MB/s ta 0:00:01
  Stored in directory: /Users/kaminarishuukobi/Library/Caches/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib
Note: you may need to restart the kernel to use updated packages.


In [3]:
from geopy.geocoders import Nominatim

#### Step 1: Transform the data in Wiki into the pandas dataframe

In [12]:
#Using beautiful soup to get the data
website_url = requests.get('https://en.wikipedia.org/wiki/Districts_of_Hong_Kong').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify())
HK_18 = soup.find('table',{'class':'wikitable sortable'})

In [68]:
columns = [th.text.replace('\n', '') for th in HK_18.find('tr').find_all('th')]

In [112]:
trs = HK_18.find_all('tr')[1:]
rows = list()
for tr in trs:
    rows.append([td.text.replace('\n', '').replace('\xa0', '') for td in tr.find_all('td')])
rows[:5]


AttributeError: 'DataFrame' object has no attribute 'find_all'

In [113]:
df_HK= pd.DataFrame(data=rows, columns=columns)

In [114]:
df_HK

,District,Chinese,Population [6],Area(km²),Density(/km²),Region
0,Central and Western,中西區,"244,600",12.44,"19,983.92",Hong Kong Island
1,Eastern,東區,"574,500",18.56,"31,217.67",Hong Kong Island
2,Southern,南區,"269,200",38.85,"6,962.68",Hong Kong Island
3,Wan Chai,灣仔區,"150,900",9.83,"15,300.10",Hong Kong Island
4,Sham Shui Po,深水埗區,"390,600",9.35,"41,529.41",Kowloon
5,Kowloon City,九龍城區,"405,400",10.02,"40,194.70",Kowloon
6,Kwun Tong,觀塘區,"641,100",11.27,"56,779.05",Kowloon
7,Wong Tai Sin,黃大仙區,"426,200",9.30,"45,645.16",Kowloon
8,Yau Tsim Mong,油尖旺區,"318,100",6.99,"44,864.09",Kowloon
9,Islands,離島區,"146,900",175.12,825.14,New Territories


#### Step 2: Data cleaning

In [115]:
# Drop useless columns
df_HK.drop(['Chinese','Region','Area(km²)','Population [6]'], axis=1,inplace=True)  

In [116]:
#Drop some distrcits that Bruce doesnt' like to open a coffee shop.
df_HK.drop([9,14,2,16,17],axis=0,inplace=True)

In [117]:
#Reset Index
df_HK = df_HK.reset_index(drop=True)

In [119]:
df_HK

,District,Density(/km²)
0,Central and Western,"19,983.92"
1,Eastern,"31,217.67"
2,Wan Chai,"15,300.10"
3,Sham Shui Po,"41,529.41"
4,Kowloon City,"40,194.70"
5,Kwun Tong,"56,779.05"
6,Wong Tai Sin,"45,645.16"
7,Yau Tsim Mong,"44,864.09"
8,Kwai Tsing,"21,503.86"
9,North,"2,220.19"


### Step 3: get longitude and latitude for each district

In [123]:
#Read CSV File that contain Longitude and latitude
LaLo = pd.read_csv('HK_LL.csv')

In [131]:
df_HK12= pd.merge(df_HK,LaLo, on = 'District')

In [127]:
df_HK12

,District,Density(/km²),Latitude,Longitude
0,Central and Western,"19,983.92",22.285345,114.150452
1,Eastern,"31,217.67",22.273280,114.233118
2,Wan Chai,"15,300.10",22.277465,114.173026
3,Sham Shui Po,"41,529.41",22.330095,114.160940
4,Kowloon City,"40,194.70",22.331013,114.190366
5,Kwun Tong,"56,779.05",22.312126,114.226501
6,Wong Tai Sin,"45,645.16",22.342063,114.194388
7,Yau Tsim Mong,"44,864.09",22.318775,114.161839
8,Kwai Tsing,"21,503.86",22.341333,114.104364
9,North,"2,220.19",22.350627,114.184916


# Using Foursqaure to see the distribution of Japanese Restaurant

In [199]:
##Foursquare
CLIENT_ID = '0PCSZ3NLJLPZKYIIEEHLZ4QTP0DSKAOGFGQYCWYKRQQ3QDM0' # your Foursquare ID
CLIENT_SECRET = 'S4ZO0COFX45U2Z150JJP11WJOQBE1EHWW1OJV15KI4FW2TDU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0PCSZ3NLJLPZKYIIEEHLZ4QTP0DSKAOGFGQYCWYKRQQ3QDM0
CLIENT_SECRET:S4ZO0COFX45U2Z150JJP11WJOQBE1EHWW1OJV15KI4FW2TDU


In [200]:
# TRY TO SEE coffee shops in Central and Western
LIMIT = 500

In [201]:
latitude = 22.28056
longitude = 114.17222

In [209]:
search_query = 'Thai Restaurant'
radius = 20000
print(search_query + ' .... OK!')

Thai Restaurant .... OK!


In [210]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url
results = requests.get(url).json()

'https://api.foursquare.com/v2/venues/search?client_id=0PCSZ3NLJLPZKYIIEEHLZ4QTP0DSKAOGFGQYCWYKRQQ3QDM0&client_secret=S4ZO0COFX45U2Z150JJP11WJOQBE1EHWW1OJV15KI4FW2TDU&ll=22.28056,114.17222&v=20180605&query=Thai Restaurant&radius=20000&limit=500'

In [212]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",False,547158de498e985ea62823b0,"2/F, W Square, 314-324 Hennessy Rd",HK,香港,香港,NaN,671,"[2/F, W Square, 314-324 Hennessy Rd, 香港]","[{'label': 'display', 'lat': 22.27841948816095...",22.278419,114.178314,NaN,NaN,Sun Thai Restaurant (新泰東南亞餐廳),v-1564219333
1,"[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",False,4b5be4cdf964a5200b1c29e3,"1/F, Wui Fu Bldg, 10-12 Stone Nullah Ln",HK,灣仔,香港,NaN,513,"[1/F, Wui Fu Bldg, 10-12 Stone Nullah Ln, 灣仔區,...","[{'label': 'display', 'lat': 22.27613718250227...",22.276137,114.173648,NaN,灣仔區,Taste Thai Restaurant and Pub (嚐泰泰國餐廳酒吧),v-1564219333
2,"[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",False,519eed36498eca59d67e45df,"2/F, De Fenwick, 42-50 Lockhart Rd",HK,灣仔,香港,NaN,305,"[2/F, De Fenwick, 42-50 Lockhart Rd, 灣仔區, 香港]","[{'label': 'display', 'lat': 22.27834435389058...",22.278344,114.170473,NaN,灣仔區,Bangkok Thai Restaurant (曼谷泰菜),v-1564219333
3,"[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",False,4ce902b6baa6a1cdf679316c,"Shop E, 3/F, Sun Kai Mansion, 38-46 Hennessy Rd",HK,灣仔,香港,NaN,361,"[Shop E, 3/F, Sun Kai Mansion, 38-46 Hennessy ...","[{'label': 'display', 'lat': 22.27784113153323...",22.277841,114.170302,NaN,灣仔區,J&K Thai Restaurant (泰食廊),v-1564219333
4,"[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",False,5a5b3f9612f0a96c741d6cf2,8 Lan Kwai Fong,HK,香港,香港,NaN,1711,"[8 Lan Kwai Fong, 香港]","[{'label': 'display', 'lat': 22.280718, 'lng':...",22.280718,114.155609,NaN,NaN,Infinity Thai Restaurant & Bar (狗毛),v-1564219333


In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

In [214]:
dataframe_filtered.name

0                         Sun Thai Restaurant (新泰東南亞餐廳)
1              Taste Thai Restaurant and Pub (嚐泰泰國餐廳酒吧)
2                        Bangkok Thai Restaurant (曼谷泰菜)
3                             J&K Thai Restaurant (泰食廊)
4                   Infinity Thai Restaurant & Bar (狗毛)
5                Pandan Leaves Thai Restaurant (香葉泰國菜館)
6                Coconut Tree Thai Restaurant (椰子樹泰國餐廳)
7                      Chilli Thai Restaurant (泰出色泰國餐廳)
8                  Orange Elephant Thai Restaurant (橙象)
9                              NARA Thai Restaurant Bar
10                       Bangkok Thai Restaurant (曼谷泰菜)
11                        Sunny Thai Restaurant (晴苑泰國菜)
12                       Bangkok Thai Restaurant (曼谷泰菜)
13             Wong Chun Chun Thai Restaurant (黃珍珍泰國菜館)
14                             Ban Thai Restaurant (泰屋)
15                     Ruamjai Thai Restaurant (同心泰國菜館)
16    Thalassic Thai Restaurant (Thalassic Thai Rest...
17                      Bonnie's Thai Restaurant

In [215]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

#From the above distribution, district with high density of restaurant will be drop and 4 distrits will remain.


In [217]:
df_HK12

,District,Density(/km²),Latitude,Longitude
0,Central and Western,"19,983.92",22.285345,114.150452
1,Eastern,"31,217.67",22.273280,114.233118
2,Wan Chai,"15,300.10",22.277465,114.173026
3,Sham Shui Po,"41,529.41",22.330095,114.160940
4,Kowloon City,"40,194.70",22.331013,114.190366
5,Kwun Tong,"56,779.05",22.312126,114.226501
6,Wong Tai Sin,"45,645.16",22.342063,114.194388
7,Yau Tsim Mong,"44,864.09",22.318775,114.161839
8,Kwai Tsing,"21,503.86",22.341333,114.104364
9,North,"2,220.19",22.350627,114.184916


In [219]:
#drop districts with high density of Thai restaurant
df_HK12.drop([0,1,2,4,6,9,7,10,11],axis=0,inplace=True)

In [220]:
df_HK12 = df_HK12.reset_index(drop=True)

In [221]:
df_HK12

,District,Density(/km²),Latitude,Longitude
0,Sham Shui Po,"41,529.41",22.330095,114.160940
1,Kwun Tong,"56,779.05",22.312126,114.226501
2,Kwai Tsing,"21,503.86",22.341333,114.104364
3,Tsuen Wan,"4,887.38",22.371661,114.113470


In [222]:
#Add Rent price
Rent = pd.read_csv('HK_rent.csv')

In [223]:
df_HK12= pd.merge(df_HK,Rent, on = 'District')

In [224]:
df_HK12

,District,Density(/km²),Rent
0,Sham Shui Po,"41,529.41",138000.0
1,Kwun Tong,"56,779.05",60000.0
2,Kwai Tsing,"21,503.86",65000.0
3,Tsuen Wan,"4,887.38",80000.0


# The Final decison: Kwai Tsing for the reason of high population density and low rent